see https://root.cern.ch/doc/v610/ApplicationClassificationKeras_8py_source.html

In [1]:
# /work/gallim/anaconda3/envs/dask-pyroot-tutorial/bin/python
import ROOT
import uproot
import awkward as ak
from array import array
import uproot3

Welcome to JupyROOT 6.22/02


In [2]:
output_dir = "/eos/home-g/gallim/www/plots/Hgg/NanoMicroCompare"
xml_model = "/work/gallim/devel/HiggsDNA_studies/tmva-to-xgboost/2017_weights/PhoID_barrel_UL2017_GJetMC_SATrain_nTree2k_LR_0p1_13052020_BDTG.weights.xml"
processed_nano = "processed_nano.root"

In [3]:
inputs = {
    "lead_energyRaw": "SCRawE", 
    "lead_r9": "r9", 
    "lead_sieie":"sigmaIetaIeta", 
    "lead_etaWidth": "etaWidth", 
    "lead_phiWidth": "phiWidth", 
    "lead_sieip": "covIEtaIPhi", 
    "lead_s4": "s4", 
    "lead_pfPhoIso03": "phoIso03", 
    "lead_pfChargedIsoPFPV": "chgIsoWrtChosenVtx", 
    "lead_pfChargedIsoWorstVtx": "chgIsoWrtWorstVtx",
    "lead_eta": "scEta", 
    "lead_fixedGridRhoAll": "rho"
}

In [4]:
#nanoaod_files_dir = "/work/gallim/devel/HiggsDNA_studies/out/test/GluGluHToGG_M125_TuneCP5_13TeV-amcatnloFXFX-pythia8"
#nanoaod_arr = ak.from_parquet(nanoaod_files_dir, columns=inputs)

In [5]:
# Write to ROOT file

"""
with uproot3.recreate(processed_nano) as f:
    branchdict = {}
    arraydict = {}
    
    for input_name in inputs.keys():
        branchdict[input_name] = str(nanoaod_arr[input_name].type.type).replace('?', '')
        arraydict[input_name] = nanoaod_arr[input_name]
    
    f["Events"] = uproot3.newtree(branchdict)
    f["Events"].extend(arraydict)
"""

'\nwith uproot3.recreate(processed_nano) as f:\n    branchdict = {}\n    arraydict = {}\n    \n    for input_name in inputs.keys():\n        branchdict[input_name] = str(nanoaod_arr[input_name].type.type).replace(\'?\', \'\')\n        arraydict[input_name] = nanoaod_arr[input_name]\n    \n    f["Events"] = uproot3.newtree(branchdict)\n    f["Events"].extend(arraydict)\n'

In [6]:
# Setup TMVA
ROOT.TMVA.Tools.Instance()
ROOT.TMVA.PyMethodBase.PyInitialize()
reader = ROOT.TMVA.Reader("Color:!Silent")

In [7]:
# Load data
data = ROOT.TFile(processed_nano)
events = data.Get("Events")

In [8]:
branches = {}

for branch in events.GetListOfBranches():
    name = branch.GetName()
    tmva_name = inputs[name]
    branches[tmva_name] = array("f", [-999])
    reader.AddVariable(tmva_name, branches[tmva_name])
    events.SetBranchAddress(name, branches[tmva_name])

In [9]:
branches

{'SCRawE': array('f', [-999.0]),
 'r9': array('f', [-999.0]),
 'sigmaIetaIeta': array('f', [-999.0]),
 'etaWidth': array('f', [-999.0]),
 'phiWidth': array('f', [-999.0]),
 'covIEtaIPhi': array('f', [-999.0]),
 's4': array('f', [-999.0]),
 'phoIso03': array('f', [-999.0]),
 'chgIsoWrtChosenVtx': array('f', [-999.0]),
 'chgIsoWrtWorstVtx': array('f', [-999.0]),
 'scEta': array('f', [-999.0]),
 'rho': array('f', [-999.0])}

In [10]:
reader.BookMVA("BDT", ROOT.TString(xml_model))

                         : Booking "BDT" of type "BDT" from /work/gallim/devel/HiggsDNA_studies/tmva-to-xgboost/2017_weights/PhoID_barrel_UL2017_GJetMC_SATrain_nTree2k_LR_0p1_13052020_BDTG.weights.xml.
                         : Reading weight file: /work/gallim/devel/HiggsDNA_studies/tmva-to-xgboost/2017_weights/PhoID_barrel_UL2017_GJetMC_SATrain_nTree2k_LR_0p1_13052020_BDTG.weights.xml
DataSetInfo              : [Default] : Added class "Signal"
DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDTG" of type: "BDT"


In [11]:
for i in range(20):
    events.GetEntry(i)
    print(reader.EvaluateMVA("BDT"))

0.9274684916811315
0.9823074531371638
0.6621159770753586
0.9932704394092093
0.9848337653997874
0.9827771474745182
0.8379285637869784
0.9870612177227733
0.9226851143983081
0.981013541882283
-0.04482493924862174
0.6301817221793595
0.9903119507607101
0.5415062747285724
-0.46862629175251747
0.9293414188886833
0.9842933760030668
0.23588399484538014
0.8165900295797033
0.9190325640256054
